In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
num_classes = 2

# Image Data Generators for augmenting and rescaling images
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [4]:
# Load training, validation, and test data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # VGG16 expects 224x224 images
    batch_size=32,
    class_mode='categorical'  # Categorical for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [5]:
# Load the VGG16 model, excluding the top fully connected layers
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

In [6]:
# Freeze the VGG16 layers so they are not updated during training
base_model.trainable = False

In [7]:
# Build the additional layers on top of the base model
x = base_model.output

In [8]:
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Use Global Average Pooling to reduce dimensionality before the dense layers
x = GlobalAveragePooling2D()(x)

# Add fully connected layers with Dropout for regularization
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

In [9]:
# Output layer with softmax activation for multi-class classification
predictions = Dense(num_classes, activation='softmax')(x)

In [10]:
# Create the final model
model = Model(
    inputs = base_model.input,
    outputs = predictions
)

In [11]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate = 0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# Display the model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 3, 3, 512)      │             

 Total params: 20,620,354 (78.66 MB)

 Trainable params: 5,903,618 (22.52 MB)

 Non-trainable params: 14,716,736 (56.14 MB)

In [13]:
number_of_epochs = 30

In [14]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs = number_of_epochs  # Adjust the number of epochs based on your dataset size
)

Epoch 1/30


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5076 - loss: 1.2009 - val_accuracy: 0.4286 - val_loss: 0.7135
Epoch 2/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5625 - loss: 0.9141

2024-11-05 22:56:33.121224: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/envs/Thesis/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 552ms/step - accuracy: 0.5625 - loss: 0.9141 - val_accuracy: 0.4286 - val_loss: 0.7089
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5268 - loss: 0.9931 - val_accuracy: 0.4286 - val_loss: 0.7121
Epoch 4/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5312 - loss: 1.0001

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 530ms/step - accuracy: 0.5312 - loss: 1.0001 - val_accuracy: 0.4286 - val_loss: 0.7252
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.4983 - loss: 1.1100 - val_accuracy: 0.4286 - val_loss: 0.7127
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 554ms/step - accuracy: 0.5938 - loss: 0.7651 - val_accuracy: 0.4286 - val_loss: 0.7085
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.5242 - loss: 0.9559 - val_accuracy: 0.4286 - val_loss: 0.7085
Epoch 8/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.6250 - loss: 0.6846

2024-11-05 22:57:39.231441: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 547ms/step - accuracy: 0.6250 - loss: 0.6846 - val_accuracy: 0.4286 - val_loss: 0.7106
Epoch 9/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6610 - loss: 0.7689 - val_accuracy: 0.4286 - val_loss: 0.7126
Epoch 10/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 545ms/step - accuracy: 0.6562 - loss: 0.6878 - val_accuracy: 0.4286 - val_loss: 0.7095
Epoch 11/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.6806 - loss: 0.7060 - val_accuracy: 0.4286 - val_loss: 0.7153
Epoch 12/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 554ms/step - accuracy: 0.5938 - loss: 0.6960 - val_accuracy: 0.4286 - val_loss: 0.7196
Epoch 13/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6126 - loss: 0.6775 - val_accuracy: 0.4643 - val_loss: 0.7039
Epoch 14/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 555ms/step - accuracy: 0.5938 - loss: 0.6972 - val_accuracy: 0.5000 - val_loss: 0.7007
Epoch 15/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.6095 - loss: 0.7847 - val_accuracy: 0.4643 - val_loss: 0.6943
Epoch

2024-11-05 22:59:08.730372: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 548ms/step - accuracy: 0.6250 - loss: 0.7235 - val_accuracy: 0.4643 - val_loss: 0.6945
Epoch 17/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.5818 - loss: 0.8336 - val_accuracy: 0.4643 - val_loss: 0.6987
Epoch 18/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 544ms/step - accuracy: 1.0000 - loss: 0.3573 - val_accuracy: 0.4643 - val_loss: 0.6954
Epoch 19/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6906 - loss: 0.6365 - val_accuracy: 0.4643 - val_loss: 0.7142
Epoch 20/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 542ms/step - accuracy: 0.7188 - loss: 0.5431 - val_accuracy: 0.4643 - val_loss: 0.7024
Epoch 21/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6015 - loss: 0.7520 - val_accuracy: 0.4643 - val_loss: 0.6879
Epoch 22/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 539ms/step - accuracy: 0.6250 - loss: 0.6409 - val_accuracy: 0.5000 - val_loss: 0.6921
Epoch 23/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.7011 - loss: 0.5445 - val_accuracy: 0.6071 - val_loss: 0.6585
Epoc

In [15]:
# Optionally, unfreeze some layers in the base model and fine-tune
base_model.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs = 10  # Fine-tune for a few more epochs
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 42s 6s/step - accuracy: 0.7342 - loss: 0.5676 - val_accuracy: 0.5714 - val_loss: 0.6566
Epoch 2/10
1/6 ━━━━━━━━━━━━━━━━━━━━ 31s 6s/step - accuracy: 0.5625 - loss: 0.9718

2024-11-05 23:02:35.564072: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 606ms/step - accuracy: 0.5625 - loss: 0.9718 - val_accuracy: 0.7143 - val_loss: 0.6402
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.8292 - loss: 0.4413 - val_accuracy: 0.7857 - val_loss: 0.6045
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 523ms/step - accuracy: 0.7188 - loss: 0.5478 - val_accuracy: 0.7143 - val_loss: 0.6061
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.8085 - loss: 0.4781 - val_accuracy: 0.6786 - val_loss: 0.6174
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 527ms/step - accuracy: 0.9062 - loss: 0.2996 - val_accuracy: 0.7500 - val_loss: 0.5665
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.7552 - loss: 0.5198 - val_accuracy: 0.7143 - val_loss: 0.5811
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 526ms/step - accuracy: 0.7812 - loss: 0.3905 - val_accuracy: 0.6071 - val_loss: 0.5691
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 39s 7s/step - accuracy: 0.7946 - loss: 0.5742 - val_accuracy: 0.7500 - val_loss: 0.5853
Epoch 10/10

In [17]:
# Save the model, optional
model.save(f'Pth_Files/VGG16_{number_of_epochs}.keras')

## Model Testing

In [18]:
def load_image_test(img_path, model):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array /= 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)
    predicted_probability = np.max(predictions) * 100  # Get the probability of the predicted class


    # If you know the class labels (e.g., from a class_indices mapping when using a data generator)
    class_labels = {0: 'Class1', 1: 'Class2', 2: 'Class3'}  # Update with your actual labels
    predicted_label = class_labels[predicted_class_index[0]]

    # Print the prediction results
    print(f"Predicted class index: {predicted_class_index[0]}")
    print(f"Predicted class label: {predicted_label}")
    print(f"Prediction confidence: {predicted_probability:.2f}%")

In [19]:
# Load your trained model
model = load_model(f'Pth_Files/VGG16_{number_of_epochs}.keras')

In [20]:
test_img_path = test_dir + '/Agaricus_abruptibulbus/Agaricus_abruptibulbus_1.jpg'
img_array = load_image_test(test_img_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
Predicted class index: 0
Predicted class label: Class1
Prediction confidence: 53.70%


In [21]:
test_img_path = test_dir + '/Amanita_excelsa/Amanita_excelsa_1.jpg'
img_array = load_image_test(test_img_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Predicted class index: 1
Predicted class label: Class2
Prediction confidence: 73.92%
